In [9]:

'''
---------------------------------------------------------------
Simulation for Function-on-Function Linear Regression
---------------------------------------------------------------
* 1 d

* 1 Response Y & 1 Predictor X   

* Sparse (num of grid: 21)

* Training samp size: 200
  Test     samp size: 100

* Using Gaussian Kernal Function in LPR

* Using Leave-One-Curve-Out CV for bandwidth selection in LPR 
---------------------------------------------------------------
'''
#####
import os
os.chdir('/home/hot50773')
import GetData
import FFReg
import Plotting
import lpr
import time
import scipy as sp
from scipy import io
os.chdir('/home/hot50773/SIM_1y1x_sparse')

###
import numpy as np
# ---------
# Settings
# ---------
# sim_rep_time = 100

tran_samp_size = 200
test_samp_size = 100
samp_size = tran_samp_size + test_samp_size

range_num_pts = np.array([2, 6])
num_pts = range_num_pts[1]

# -------------
# Setting of X
# -------------
def X_Mean_Func(s):
    #input, output: vector
    return(s /  + 2 * np.sin(s))

def Eigen_Funcs_X(s):
    #input  :d-vector
    #output:array with dim (2 * d)
    return(np.array([np.sqrt(2 / 5) * -np.cos(2 * np.pi * s / 5), 
                     np.sqrt(2 / 5) *  np.sin(2 * np.pi * s / 5),
                     np.sqrt(2 / 5) * -np.cos(4 * np.pi * s / 5)]))

num_grid_X = 21
range_X = np.array([0, 5])
X_eigen_val = np.array([9, 4, 1])

# -------------
# Setting of Y
# -------------

def Y_Mean_Func(s):
    #input, output: vector
    return(s / 2 + np.sin(s))

def Eigen_Funcs_Y(s):
    #input  :d-vector
    #output:array with dim (2 * d)
    return(np.array([np.sqrt(2 / 5) * -np.cos(np.pi * s / 5), 
                     np.sqrt(2 / 5) * np.sin(np.pi * s / 5),
                     np.sqrt(2 / 5) * -np.cos(2 * np.pi * s / 5)])) 

num_grid_Y = 21

range_Y = np.array([0, 5])

B = np.array([[1, 0.8, 0.8], [-1, 0.5, -0.5],[0.8, -0.5, 0.2]])

# bandwidth
X_candidate_h_mean = np.arange(0.3, 0.7, 0.05).reshape(-1, 1)
X_candidate_h_cov = np.arange(0.3, 0.8, 0.05).repeat(2).reshape(-1, 2)
X_candidate_h_diag_cov = np.arange(0.3, 0.8, 0.05).reshape(-1, 1)

Y_candidate_h_mean = np.arange(0.3, 0.8, 0.05).reshape(-1, 1)
Y_candidate_h_cov = np.arange(0.3, 0.8, 0.05).repeat(2).reshape(-1, 2)

X_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.3, 0.8, 6), 
                                                 np.linspace(0.3, 0.8, 6))).T.reshape(-1,2)

In [10]:
name = ['X', 'Y']

In [11]:
sim_rep_time = 100

In [7]:
# #Generate and save data

# np.random.seed(104)
# X_data = []
# Y_data = []
# for i in range(sim_rep_time):
#     X_data.append(GetData.Get_X_data(Mean_Func  = X_Mean_Func,
#                                      Eigen_Funcs = Eigen_Funcs_X, 
#                                      eigen_val  = X_eigen_val, 
#                                      domain = range_X, 
#                                      tran_size  = tran_samp_size, 
#                                      test_size  = test_samp_size,
#                                      num_grid = num_grid_X,
#                                      sparse  = True,
#                                      num_pts = range_num_pts))


#     #Generate Simulation data of Y

#     Y_data.append(GetData.Get_Y_data(Mean_Func = Y_Mean_Func, 
#                                      Eigen_Funcs = Eigen_Funcs_Y,
#                                      X_fpcs = [X_data[i].fpcs],
#                                      B = [B], 
#                                      domain = range_Y, 
#                                      test_size = test_samp_size, 
#                                      num_grid = num_grid_Y,
#                                      sparse  = True,
#                                      num_pts = range_num_pts))
    
#     #Save data
#     # for 'refund' in R
#     datas = [X_data[i], Y_data[i]]
#     for j in range(2):
#         np.savetxt('tran '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].obs_tran)), 
#                    newline='\n' ,delimiter = ' ', fmt="%s")
        
#         np.savetxt('test '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].obs_test)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('real_on_grid '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].real_val_on_grid)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('tran_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].tran_time_pts)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('test_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].test_time_pts)), 
#                    newline='\n', delimiter = ' ', fmt="%s")

#     #for 'PACE' in Matlab
#     X_sim_data = {'t': np.bmat([[X_data[i].tran_time_pts],[X_data[i].test_time_pts]]), 
#                   'obs': np.bmat([[X_data[i].obs_tran],[X_data[i].obs_test]]), 
#                   'fpcs': X_data[i].fpcs}
#     Y_sim_data = {'t': np.bmat([[Y_data[i].tran_time_pts],[Y_data[i].test_time_pts]]), 
#                   'obs': np.bmat([[Y_data[i].obs_tran],[Y_data[i].obs_test]]), 
#                   'fpcs': Y_data[i].fpcs,
#                   'real_test_Y': Y_data[i].real_val_on_grid[tran_samp_size:,]}
    
#     sp.io.savemat('d:/Work_Jupyter/敏/SIM_1y1x_sparse/sim_X_data_'+str(i+1)+'.mat', X_sim_data)
#     sp.io.savemat('d:/Work_Jupyter/敏/SIM_1y1x_sparse/sim_Y_data_'+str(i+1)+'.mat', Y_sim_data)
    
# sp.io.savemat('d:/Work_Jupyter/敏/SIM_1y1x_sparse/sim_X_time_grid.mat',
#                 {'grid': X_data[0].time_grid})
# sp.io.savemat('d:/Work_Jupyter/敏/SIM_1y1x_sparse/sim_Y_time_grid.mat',
#                 {'grid': Y_data[0].time_grid})



In [12]:
X_time_grid = sp.io.loadmat('/home/hot50773/SIM_1y1x_sparse/sim_X_time_grid.mat')['grid'][0]
Y_time_grid = sp.io.loadmat('/home/hot50773/SIM_1y1x_sparse/sim_Y_time_grid.mat')['grid'][0]

In [ ]:
my_MISE = np.zeros(sim_rep_time)
# my_MaxAbs = np.zeros(sim_rep_time)
X_data = []
Y_data = []
start_time = time.time()
for i in range(sim_rep_time):
    #load data from mat
    X_sim_data_i = sp.io.loadmat('/home/hot50773/SIM_1y1x_sparse/sim_X_data_'+str(i+1)+'.mat')
    Y_sim_data_i = sp.io.loadmat('/home/hot50773/SIM_1y1x_sparse/sim_Y_data_'+str(i+1)+'.mat')
    
    #convert into our data form
    X_data.append(GetData.Load_Data(sim_data = X_sim_data_i,time_grid=X_time_grid))
    Y_data.append(GetData.Load_Data(sim_data = Y_sim_data_i,time_grid=Y_time_grid, response = True))
    
    '''
    Results of X
    '''

    result_X = FFReg.Get_FPCA_Result(X_data[i], X_candidate_h_mean, 
                                      X_candidate_h_cov, X_candidate_h_diag_cov, 
                                      ker_fun = 'Gaussian')
    #fitting test sample
    fit_test_X = result_X.Restruect_Fun(X_data[i].test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X_data[i].obs_test)[1]

    '''
    Results of Y
    '''


    result_Y = FFReg.Fit_Mean_and_Cov(Y_data[i], Y_candidate_h_mean, 
                                      Y_candidate_h_cov, ker_fun = 'Gaussian')


    '''
    Construct Coefficient Function

        1. fit cov_Xi_Xj
        2. fit cov_Xi_Y
        3. combine them to build coefficient function

    '''
    fit_cov_XY = FFReg.Fit_Cov_XY(X_time_pts = X_data[i].tran_time_pts,
                                    Y_time_pts = Y_data[i].tran_time_pts,
                                    obs_X = X_data[i].obs_tran, 
                                    obs_Y = Y_data[i].obs_tran,
                                    X_time_grid = X_data[i].time_grid,
                                    Y_time_grid = Y_data[i].time_grid, 
                                    fit_X_mean = result_X.mean_fun, 
                                    fit_Y_mean = result_Y.mean_fun, 
                                    candidate_h_cov = X_Y_candidate_h_cov, 
                                    ker_fun = 'Gaussian')

    '''
    Coefficient Function
    '''

    fit_cov_XX = result_X.cov_fun

    inv_fit_cov_XX = np.linalg.pinv(fit_cov_XX, rcond= 0.07)

    fit_Beta_without_delta =  np.matmul(inv_fit_cov_XX,fit_cov_XY)

    ## add delta
    if i == 0:
        delta = np.diag(np.repeat([X_data[i].time_grid[1] - X_data[i].time_grid[0]],
                                   [num_grid_X]))
        delta_inv = np.linalg.inv(delta)

    fit_Beta  = np.matmul(delta_inv, fit_Beta_without_delta)


    #---------------------------------------
    # fit Response of test set on grid
    #---------------------------------------
    fit_test_X_center = fit_test_X - result_X.mean_fun


    #compute test_Y(on grid pts)
    fit_test_Y = result_Y.mean_fun + np.matmul(fit_Beta_without_delta.T, fit_test_X_center.T).T


    #real value of test_Y (on grid pts)
    
    real_test_Y = Y_data[i].real_val_on_grid[tran_samp_size:,:]
#     if i <= 2:
#         Plotting.Plot_Restruct_Funcs(fit_X = np.asarray(fit_test_Y),
#                                      real_X= Y_data[i].real_val_on_grid[tran_samp_size:],
#                                      time_grid = Y_data[i].time_grid,
#                                      obs_X= Y_data[i].obs_test,
#                                      obs_time=Y_data[i].test_time_pts,
#                                      num_plot= 4,
#                                      y_lim= [-10,10]
#                                      ,legend_location = 3)

    #========================================================
    #compute RMSPE (relative mean squared prediction error)
    #========================================================
    grid_width = (Y_data[i].time_grid[1] - Y_data[i].time_grid[0])
    #======================================
    #compute Mean integrated squared error
    #======================================

    my_MISE[i] = np.sum(np.square(real_test_Y - fit_test_Y) * grid_width, 1).mean()

    # maxima of absolute error
#     my_MaxAbs[i] = np.abs(real_test_Y - fit_test_Y).max()
    
print(time.time()-start_time)



In [1]:
import scipy as sp
from scipy import io
import numpy as np

In [6]:
PACE_result = sp.io.loadmat('D:/Work_Jupyter/敏/SIM_1y1x_sparse/result_PACE_sparse_85.mat')

In [8]:
np.savetxt('MISE_sparse_real_loocv', my_MISE, newline='\n', delimiter = ' ', fmt="%s")

In [4]:
my_MISE = np.loadtxt('MISE_sparse_real_loocv', delimiter = ' ')

In [7]:
# create the quantile value of each array
# save in tuple
quantile_PACE_MISE = []
quantile_PACE_RMSPE = []
quantile_my_MISE = []
quantile_my_RMSPE = []



#add mean        
quantile_PACE_MISE.append(np.asscalar(np.mean(PACE_result['MISE'])))
        
quantile_my_MISE.append(np.asscalar(np.mean(my_MISE)))
#add median
    
quantile_PACE_MISE.append(np.asscalar(np.median(PACE_result['MISE'])))

quantile_my_MISE.append(np.asscalar(np.median(my_MISE)))
#add std

quantile_PACE_MISE.append(np.asscalar(np.std(PACE_result['MISE'])))

quantile_my_MISE.append(np.asscalar(np.std(my_MISE)))


quantile_PACE_MISE = tuple(quantile_PACE_MISE)
quantile_my_MISE = tuple(quantile_my_MISE)

In [8]:
# print table
# using lpr_gpu
# compare MISE of 'PACE'(from matlab) and FFReg
print("====================================================")
print("      Compare Mean Integrated Squared Error         ")
print("====================================================")
print("|      method    |   mean   |  median  |    std    |")
print("----------------------------------------------------")
print(" PACE in matlab" ,3 * " %10.3f"%quantile_PACE_MISE[:3])
print("----------------------------------------------------")
print("   my  method  " ,3 * " %10.3f"%quantile_my_MISE[:3])
print("====================================================")
# FVE of "PACE in matlab" = 85 % 

      Compare Mean Integrated Squared Error         
|      method    |   mean   |  median  |    std    |
----------------------------------------------------
 PACE in matlab      13.764     10.069     10.168
----------------------------------------------------
   my  method         9.599      8.823      4.505
